<a href="https://colab.research.google.com/github/MarMarhoun/freelance_work/blob/main/up_work/um_translation/t5_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train and Fine tune T5 model for traduction task:

In [ ]:
!pip install transformers
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.0 MB/s eta 0:00:00


# Testing code

In [ ]:
import torch
import pyarrow as pa
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
# Mount Google Drive

# Load the dataset from the CSV file stored in Google Drive
path = '/content/train_set.csv'
path_val = '/content/dev_set.csv'
path_test = '/content/test_ur.csv'


ds_train = pd.read_csv(path)
ds_val = pd.read_csv(path_val)
ds_test = pd.read_csv(path_test)

# Drop rows with missing values
ds_train = ds_train.dropna()
ds_val = ds_val.dropna()
ds_test = ds_test.dropna()

train_df = ds_train
val_df = ds_val
test_df = ds_test

# Create a DatasetDict object
datasets = DatasetDict()

# Convert train_df to a pyarrow.Table object
train_table = pa.Table.from_pandas(train_df)
val_table = pa.Table.from_pandas(val_df)
#val_table = pa.Table.from_pandas(val_df)
# Create the train dataset
datasets['train'] = Dataset(train_table)

# Create the validation and test datasets
datasets['validation'] = Dataset(val_table)
#datasets['test'] = Dataset(test_table)

# Load the pre-trained tokenizer and model
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Define the preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples['Urdu'], truncation=True)
    labels = tokenizer(examples['English'], truncation=True)
    inputs['labels'] = labels['input_ids']
    return inputs



In [ ]:


def preprocess_function(examples):
    inputs = tokenizer(examples['Urdu'], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(examples['English'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = labels['input_ids']
    return inputs
# Set the format of the input data
datasets.set_format(type="torch")

# Apply the preprocessing function to the datasets
datasets = datasets.map(preprocess_function, batched=True)


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

In [ ]:

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs


)

# Create the Trainer instance
trainer = Trainer(
    model=model,                         # the instantiated Transformers model
    args=training_args,                  # training arguments
    train_dataset=datasets['train'],     # training dataset
    eval_dataset=datasets['validation']  # evaluation dataset
)



In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,4.468800
1000,1.457200


TrainOutput(global_step=1125, training_loss=2.793766370985243, metrics={'train_runtime': 350.526, 'train_samples_per_second': 51.351, 'train_steps_per_second': 3.209, 'total_flos': 609038106624000.0, 'train_loss': 2.793766370985243, 'epoch': 3.0})

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
500,1.308900
1000,1.284900
1500,1.279800


TrainOutput(global_step=1875, training_loss=1.2872889811197916, metrics={'train_runtime': 597.183, 'train_samples_per_second': 50.236, 'train_steps_per_second': 3.14, 'total_flos': 1015063511040000.0, 'train_loss': 1.2872889811197916, 'epoch': 5.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
#print(f"Validation Loss: {eval_results['eval_loss']:.4f}, Validation Accuracy: {eval_results['eval_accuracy']*100:.2f}%")
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.6193254590034485, 'eval_runtime': 1.1714, 'eval_samples_per_second': 182.68, 'eval_steps_per_second': 3.415, 'epoch': 3.0}


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
#print(f"Validation Loss: {eval_results['eval_loss']:.4f}, Validation Accuracy: {eval_results['eval_accuracy']*100:.2f}%")
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.5764871835708618, 'eval_runtime': 1.1411, 'eval_samples_per_second': 187.544, 'eval_steps_per_second': 3.505, 'epoch': 5.0}


In [ ]:
# Save the model
model.save_pretrained('t5-small-urdu-english')


In [ ]:
# Load the trained model and tokenizer
model_path = '/content/t5-small-urdu-english'
model = T5ForConditionalGeneration.from_pretrained(model_path)
# Load the pre-trained tokenizer and model
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# Define a test sentence in Urdu
test_sentence = "کیا ایک کتاب کے اثران سوچیا ہے؟"

# Tokenize the test sentence
input_ids = tokenizer.encode(test_sentence, return_tensors='pt')

# Generate a prediction
output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)

# Decode the prediction
prediction = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the prediction
print(prediction)

And if you are not a savage, then you will be able to reclaim the heavenly.


In [ ]:
output

tensor([[    0,   275,     3,    99,    25,    33,    59,     3,     9,     3,
             7,     9, 11515,     3,     6,   258,    25,    56,    36,     3,
           179,    12,     3,  1544,     3,     5,     1]])

In [ ]:
# Testing section using your testing dataset

test_table = pa.Table.from_pandas(test_df)
datasets['test'] = Dataset(test_table)

def preprocess_fct_test(examples):
    inputs = tokenizer(examples['Urdu'], truncation=True, padding='max_length', max_length=128)
    return inputs
# Set the format of the input data
datasets.set_format(type="torch")

# Apply the preprocessing function to the datasets
datasets['test'] = datasets['test'].map(preprocess_fct_test, batched=True)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# Create decoder input embeddings
decoder_inputs_embeds = tokenizer(test_sentence, return_tensors='pt')['input_ids']

# Use decoder_inputs_embeds in trainer.predict
test_results = trainer.predict(datasets['test'])#, decoder_inputs_embeds=decoder_inputs_embeds)
# Testing
#test_results = trainer.predict(datasets['test'])
predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)
labels = tokenizer.batch_decode(test_results.label_ids, skip_special_tokens=True)



In [ ]:
# Save the predictions to a CSV file
test_df['Predicted_English'] = predictions
test_df.to_csv('/content/test_results.csv', index=False)

# Display the first 5 predictions
test_df.head()

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(labels, predictions)
print(f"Test Accuracy: {accuracy*100:.2f}%")# Display the classification report
print(classification_report(labels, predictions))

# Display the confusion matrix
confusion_mtx = confusion_matrix(labels, predictions)
print(confusion_mtx)

# Normalize the confusion matrix
confusion_mtx_norm = confusion_mtx / np.sum(confusion_mtx, axis=1)[:, np.newaxis]
print(confusion_mtx_norm)

NameError: name 'accuracy_score' is not defined

## Fine-tunig section:

In [ ]:
# Fine-tuning section
# Define the fine-tuning arguments
fine_tuning_args = TrainingArguments(
    output_dir='./fine_tuning_results',  # output directory
    num_train_epochs=5,                  # total # of fine-tuning epochs
    per_device_train_batch_size=16,     # batch size per device during fine-tuning
    per_device_eval_batch_size=64,      # batch size for evaluation
    warmup_steps=500,                    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # strength of weight decay
    logging_dir='./fine_tuning_logs',    # directory for storing logs
    evaluation_strategy='epoch',         # evaluation strategy
    save_strategy='epoch',               # save strategy
    load_best_model_at_end=True,         # load the best model at the end of fine-tuning
    metric_for_best_model='eval_loss',   # metric for best model
    greater_is_better=False,             # whether the metric is better when it's greater
    save_total_limit=2,                 # total limit of saved models
    save_on_each_node=True,             # save on each node
    prediction_loss_only=True,          # prediction loss only
)

In [ ]:
# Fine-tune the model
trainer.train(datasets['train'], eval_dataset=datasets['validation'])

# Evaluate the fine-tuned model
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}, Validation Accuracy: {eval_results['eval_accuracy']*100:.2f}%")

# Save the fine-tuned model
model.save_pretrained('t5-small-urdu-english-fine-tuned')

# Prediction section
# Define the function for prediction
def prediction_function(batch):
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        labels = batch['labels']
        predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        return {'predictions': predictions, 'labels': labels}

# Predict the English translations for the test dataset
predictions = datasets['test'].map(prediction_function, batched=True)
predictions.set_format('pandas')
predictions_df = predictions.to_pandas()
predictions_df.to_csv('predictions.csv', index=False)
print("Predictions saved to 'predictions.csv'.")

# Print the evaluation metrics
from datasets import load_metric

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions, axis=1)
    metric = load_metric('accuracy')
    return metric.compute(predictions=predictions, references=labels)

metrics = compute_metrics(predictions_df[['predictions', 'labels']].values)
print(f"Test Accuracy: {metrics['accuracy']*100:.2f}%")print(classification_report(predictions_df['labels'], predictions_df['predictions']))
print(confusion_matrix(predictions_df['labels'], predictions_df['predictions']))

# Advanced code

In [ ]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils.clip_grad import clip_grad_norm_
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import os
import random

# Set the seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths to the datasets
path = "/content/drive/My Drive/T5/train_set.csv"
path_val = "/content/drive/My Drive/T5/dev_set.csv"
path_test = "/content/drive/My Drive/T5/test_ur.csv"

ds_train = pd.read_csv(path)
ds_val = pd.read_csv(path_val)
ds_test = pd.read_csv(path_test)
ds_train

ds_train=ds_train.dropna()
ds_val=ds_val.dropna()
ds_test=ds_test.dropna()


train_df = ds_train
val_df = ds_val
test_df = ds_test


# Create a custom dataset class
class UrduEnglishDataset(Dataset):
    def __init__(self, df, tokenizer, max_source_length, max_target_length):
        self.df = df
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        urdu_text = self.df.iloc[idx, 0]
        english_text = self.df.iloc[idx, 1]

        encoding = self.tokenizer.encode_plus(
            urdu_text,
            add_special_tokens=True,
            max_length=self.max_source_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        labels = self.tokenizer.encode_plus(
            english_text,
            add_special_tokens=True,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels['input_ids'].flatten(),
            'labels_attention_mask': labels['attention_mask'].flatten()
        }

# Set the model parameters
model_params = {
    "MODEL": "t5-small",
    "TRAIN_BATCH_SIZE": 2,# 4, # 8,
    "VALID_BATCH_SIZE":1, #1 2, #4,
    "TRAIN_EPOCHS": 5, ##3
    "VAL_EPOCHS": 1,
    "LEARNING_RATE": 1e-4,
    "MAX_SOURCE_TEXT_LENGTH": 64, #128, #256, # 512,
    "MAX_TARGET_TEXT_LENGTH": 64, #128, #/ 256, # 512,
    "SEED": 42,
    "GRAD_CLIP": 1.0,
    "PATIENCE": 5
}

# Set the device (TPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:

# Create the datasets and data loaders
train_dataset = UrduEnglishDataset(train_df, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"])
val_dataset = UrduEnglishDataset(val_df, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"])
test_dataset = UrduEnglishDataset(test_df, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"])

train_loader = DataLoader(train_dataset, batch_size=model_params["TRAIN_BATCH_SIZE"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=model_params["VALID_BATCH_SIZE"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=model_params["VALID_BATCH_SIZE"], shuffle=False)

# Load the model
model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])

# Set the training arguments
training_args = TrainingArguments(
    output_dir='results_t5small',
    num_train_epochs=model_params["TRAIN_EPOCHS"],
    per_device_train_batch_size=model_params["TRAIN_BATCH_SIZE"],
    per_device_eval_batch_size=model_params["VALID_BATCH_SIZE"],
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,
    eval_strategy='steps',
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    save_total_limit=5,
    report_to='tensorboard',
    learning_rate=model_params["LEARNING_RATE"],
    fp16=True
)
def compute_metrics(pred, label_ids=None):
    # Convert the label_ids to a tensor
    label_ids = label_ids if label_ids is not None else torch.tensor(pred.label_ids)

    # Convert the predictions to a tensor
    predictions = torch.tensor(pred.predictions)

    # Ensure the predictions tensor has the expected sequence length
    if predictions.size(3) != model_params["MAX_TARGET_TEXT_LENGTH"]:
        # Reshape the predictions tensor to match the expected length
        predictions = predictions.reshape(predictions.size(0), predictions.size(1), model_params["MAX_TARGET_TEXT_LENGTH"])

    # Compute the accuracy
    accuracy = torch.sum(label_ids == predictions.argmax(-1)).item()

    # Return the accuracy
    return {"accuracy": accuracy}
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics #*lambda pred: {"accuracy": torch.sum(pred.label_ids == pred.predictions.argmax(-1)).item()}
)

# Train the model
history = trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step,Training Loss,Validation Loss


<ipython-input-4-68426953da61>:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  predictions = torch.tensor(pred.predictions)


ValueError: expected sequence of length 32128 at dim 3 (got 512)

In [ ]:

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}, Validation Accuracy: {eval_results['eval_accuracy']*100:.2f}%")

# Save the model
trainer.save_model("t5small_model")

# Generate predictions for the test set
test_results = trainer.predict(test_loader)
preds = torch.tensor([x.predictions for x in test_results])
preds = torch.argmax(preds, dim=-1)

# Evaluate the predictions
def evaluate_predictions(preds, labels):
    accuracy = torch.sum(preds == labels).item() / labels.numel()
    return {"accuracy": accuracy * 100}

eval_preds = evaluate_predictions(preds, torch.tensor(test_df['labels']))
print(f"Test Accuracy: {eval_preds['accuracy']:.2f}%")# Perform inference
def do_correction(text, model, tokenizer):
    input_text = f"translate Urdu to English: {text}"
    encoding = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=model_params["MAX_SOURCE_TEXT_LENGTH"],
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_return_sequences=1,
        max_length=model_params["MAX_TARGET_TEXT_LENGTH"],
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    correction = tokenizer.decode(output[0], skip_special_tokens=True)
    return correction

input_text = "کیا ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟"
output_text = do_correction(input_text, model, tokenizer)
print(f"Corrected text: {output_text}")


Step,Training Loss,Validation Loss


ValueError: expected sequence of length 32128 at dim 3 (got 512)

In [ ]:

# Fine-tuning
def fine_tune(model, train_loader, val_loader, model_params, training_args):
    # Initialize the optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=model_params["LEARNING_RATE"])

    # Initialize the learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=model_params["PATIENCE"])

    # Initialize the best validation loss
    best_val_loss = float('inf')

    # Initialize the epoch counter
    epoch_counter = 0

    # Train the model
    for epoch in range(training_args.num_train_epochs):
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            labels_attention_mask = batch['labels_attention_mask'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                label_attention_mask=labels_attention_mask
            )

            loss = outputs.loss
            train_loss += loss.item()
            loss.backward()
            clip_grad_norm_(model.parameters(), model_params["GRAD_CLIP"])
            optimizer.step()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                labels_attention_mask = batch['labels_attention_mask'].to(device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels,
                    label_attention_mask=labels_attention_mask
                )

                loss = outputs.loss
                val_loss += loss.item()
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1)
                val_accuracy += torch.sum(preds == labels).item()

        val_loss /= len(val_loader)
        val_accuracy /= len(val_dataset)

        print(f"Epoch {epoch_counter + 1}, Training Loss: {train_loss}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_params["SAVE_PATH"])
            print("Saved the best model!")

        scheduler.step(val_loss)

        epoch_counter += 1

        if epoch_counter >= training_args.num_train_epochs:
            break

input_texts = [
    "کیا ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟",
    "میں نے یک دوست کی یادگاری کر سکی ہے؟",
    "کیا یہ پاست بھیجا گیا ہے؟"
]
labels = [
    "کیا ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟ یا نہیں ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟",
    "میں نے یک دوست کی یادگاری کر سکی ہے؟ یا نہیں ایک دوست کی یادگاری کر سکی ہے؟",
    "کیا یہ پاست بھیجا گیا ہے؟ یا نہیں یہ پاست بھیجا گیا ہے؟"
]

# Prepare the training dataset
train_dataset = prepare_dataset(input_texts, labels)
train_loader = DataLoader(train_dataset, batch_size=model_params["BATCH_SIZE"], shuffle=True)

# Prepare the validation dataset
val_dataset = prepare_dataset(input_texts, labels)
val_loader = DataLoader(val_dataset, batch_size=model_params["BATCH_SIZE"], shuffle=True)

# Fine-tune the model
fine_tune(model, train_loader, val_loader, model_params, training_args)

# Load the best model
model.load_state_dict(torch.load(model_params["SAVE_PATH"]))

# Evaluate the model
def evaluate(model, test_loader, device):
    model.eval()
    test_loss = 0.0
    test_accuracy = 0.0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            labels_attention_mask = batch['labels_attention_mask'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                label_attention_mask=labels_attention_mask
            )

            loss = outputs.loss
            test_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            test_accuracy += torch.sum(preds == labels).item()

    test_loss /= len(test_loader)
    test_accuracy /= len(test_dataset)

    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

input_texts = [
    "کیا ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟",
    "میں نے یک دوست کی یادگاری کر سکی ہے؟",
    "کیا یہ پاست بھیجا گیا ہے؟"
]
labels = [
    "کیا ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟ یا نہیں ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟",
    "میں نے یک دوست کی یادگاری کر سکی ہے؟ یا نہیں ایک دوست کی یادگاری کر سکی ہے؟",
    "کیا یہ پاست بھیجا گیا ہے؟ یا نہیں یہ پاست بھیجا گیا ہے؟"
]

# Prepare the test dataset
test_dataset = prepare_dataset(input_texts, labels)
test_loader = DataLoader(test_dataset, batch_size=model_params["BATCH_SIZE"], shuffle=False)

# Evaluate the model
evaluate(model, test_loader, device)

# Generate predictions for the test set
test_results = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        for pred, input_id in zip(preds, input_ids):
            text = tokenizer.decode(input_id.cpu().numpy(), skip_special_tokens=True)
            test_results.append((text, pred.item()))

# Save the predictions
output_df = pd.DataFrame(test_results, columns=['input_text', 'predicted_label'])
output_df.to_csv('test_predictions.csv', index=False)

# Perform inference
def do_correction(text, model, tokenizer):
    input_text = f"translate Urdu to English: {text}"
    encoding = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=model_params["MAX_SOURCE_TEXT_LENGTH"],
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        num_return_sequences=1,
        max_length=model_params["MAX_TARGET_TEXT_LENGTH"],
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    correction = tokenizer.decode(output[0], skip_special_tokens=True)
    return correction

input_text = "کیا ایک سیل اسٹاتوں میں مکمل ہوتا ہے؟"
output_text = do_correction(input_text, model, tokenizer)
print(f"Corrected text: {output_text}")